In [1]:
from typing import List

import sqlite3 as sqlite
import pandas as pd
import json

#Função para adicionar a tabela no schema
def schema_update(schema_type : str, name : str, tbl_name : str, rootpage : int, sql : str):
    cursor.execute("INSERT INTO test_analytics_engineer VALUES (?,?,?,?,?);",
                       [schema_type, name, tbl_name, rootpage, sql])

#Acessando o banco de dados
conn = sqlite.connect('clicksign_test.db')
cursor = conn.cursor()

#Criando a tabela
#Para criar a tabela player_attributes_modified diretamente por sql eu tentei:
    # create table as...
    # insert into...
    # operadoradores como -> para acessar o dado em json... 
    # A função json_extract()
#Em todas elas o script apresentava um erro na row 373 em diante e dando o erro "OperationalError: malformed JSON"
#Nesta caso, por questão de tempo decedi estrair as colunas do jsono em python, criar a tabela e executar o que foi pedido entrea tabela auxiliar player_attributes_columns e player.

#Modelo da tabela Player_Attributes_Modified
schema_type : str = 'table'
name : str = 'table'
tbl_name : str = 'player_attributes_modified'
rootpage : int = 8
sql : str = 'CREATE TABLE'
schema_update(schema_type, name, tbl_name, rootpage, sql)

#importando arquivo csv
player_attributes : pd.DataFrame = pd.read_csv('../csv_files/Player_Attributes.csv',index_col = 0) 
    
#Transformando a coluna json em colunas
row_list : List = []
for i in range(0, player_attributes.shape[0]):
    row_list.append(json.loads(player_attributes.Player_Attributes[i]))
player_attributes_columns = pd.DataFrame(row_list)

#Criando a tabela player_attributes_columns
cursor.execute("""DROP TABLE IF EXISTS player_attributes_columns;""")
cursor.execute("""
    CREATE TABLE player_attributes_columns(
        id, player_fifa_api_id, player_api_id, date, overall_rating, potential, preferred_foot,
        attacking_work_rate, defensive_work_rate, crossing, finishing, heading_accuracy, short_passing,
        volleys, dribbling, curve, free_kick_accuracy, long_passing, ball_control, acceleration, sprint_speed,
        agility, reactions, balance, shot_power, jumping, stamina, strength, long_shots, aggression,
        interceptions, positioning, vision, penalties, marking, standing_tackle, sliding_tackle, 
        gk_diving, gk_handling, gk_kicking, gk_positioning, gk_reflexes); """)

insert_query = '''INSERT INTO player_attributes_columns VALUES(
                    ?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,
                    ?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?
                 );'''
cursor.executemany(insert_query, player_attributes_columns.values.tolist())

#Criando a tabela player_attributes_modified
cursor.execute("""DROP TABLE IF EXISTS player_attributes_modified;""")
cursor.execute("""
    CREATE TABLE player_attributes_modified AS
    SELECT 
        player.id as player_id,
        player.player_name, 
        player.birthday, 
        player.height, 
        player.weight,
        player_attributes_columns.id as player_attributes_id, 
        player_attributes_columns.player_api_id, 
        player_attributes_columns.player_fifa_api_id,
        player_attributes_columns.acceleration, 
        player_attributes_columns.aggression, 
        player_attributes_columns.agility, 
        player_attributes_columns.attacking_work_rate, 
        player_attributes_columns.balance,
        player_attributes_columns.ball_control, 
        player_attributes_columns.crossing, 
        player_attributes_columns.curve, 
        player_attributes_columns.date, 
        player_attributes_columns.defensive_work_rate, 
        player_attributes_columns.dribbling,
        player_attributes_columns.finishing, 
        player_attributes_columns.free_kick_accuracy, 
        player_attributes_columns.gk_diving, 
        player_attributes_columns.gk_handling, 
        player_attributes_columns.gk_kicking,
        player_attributes_columns.gk_positioning, 
        player_attributes_columns.gk_reflexes, 
        player_attributes_columns.heading_accuracy, 
        player_attributes_columns.interceptions,
        player_attributes_columns.jumping, 
        player_attributes_columns.long_passing, 
        player_attributes_columns.long_shots, 
        player_attributes_columns.marking, 
        player_attributes_columns.overall_rating,
        player_attributes_columns.penalties, 
        player_attributes_columns.positioning, 
        player_attributes_columns.potential, 
        player_attributes_columns.preferred_foot, 
        player_attributes_columns.reactions, 
        player_attributes_columns.short_passing,
        player_attributes_columns.shot_power, 
        player_attributes_columns.sliding_tackle, 
        player_attributes_columns.sprint_speed, 
        player_attributes_columns.stamina, 
        player_attributes_columns.standing_tackle, 
        player_attributes_columns.strength,
        player_attributes_columns.vision, 
        player_attributes_columns.volleys
    FROM player
    INNER JOIN player_attributes_columns
        on player.player_fifa_api_id = cast(player_attributes_columns.player_fifa_api_id as integer)
""")

cursor.executescript('DROP TABLE IF EXISTS player_attributes_columns;')

#Modelo da tabela Player_Attributes_Modified
tbl_name : str = 'team_attributes_modified'
rootpage : int = 9
sql : str = 'CREATE TABLE'
schema_update(schema_type, name, tbl_name, rootpage, sql)

#importando arquivo csv
team_attributes : pd.DataFrame = pd.read_csv('../csv_files/Team_Attributes.csv',index_col = 0) 
    
#Transformando a coluna json em colunas
row_list : List = []
for i in range(0, team_attributes.shape[0]):
    row_list.append(json.loads(team_attributes.Team_Attributes[i]))
team_attributes_columns = pd.DataFrame(row_list)

cursor.execute("""DROP TABLE IF EXISTS team_attributes_columns;""")
cursor.execute("""
    CREATE TABLE team_attributes_columns(
        id, team_fifa_api_id, team_api_id, date, buildUpPlaySpeed, buildUpPlaySpeedClass, buildUpPlayDribbling,
        buildUpPlayDribblingClass, buildUpPlayPassing, buildUpPlayPassingClass, buildUpPlayPositioningClass,
        chanceCreationPassing, chanceCreationPassingClass, chanceCreationCrossing, chanceCreationCrossingClass,
        chanceCreationShooting, chanceCreationShootingClass, chanceCreationPositioningClass, defencePressure,
        defencePressureClass, defenceAggression, defenceAggressionClass, defenceTeamWidth, defenceTeamWidthClass, 
        defenceDefenderLineClass
        ); """)

insert_query = '''INSERT INTO team_attributes_columns VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?);'''
cursor.executemany(insert_query, team_attributes_columns.values.tolist())


#Criando a tabela team_attributes_modified
cursor.execute("""DROP TABLE IF EXISTS team_attributes_modified;""")
cursor.execute("""
    CREATE TABLE team_attributes_modified AS
    SELECT 
        team.id as team_id,  
        team.team_long_name,
        team.team_short_name, 
        team_attributes_columns.id as team_attributes_id, 
        team_attributes_columns.team_fifa_api_id, 
        team_attributes_columns.team_api_id, 
        team_attributes_columns.date, 
        team_attributes_columns.buildUpPlaySpeed, 
        team_attributes_columns.buildUpPlaySpeedClass, 
        team_attributes_columns.buildUpPlayDribbling,
        team_attributes_columns.buildUpPlayDribblingClass, 
        team_attributes_columns.buildUpPlayPassing, 
        team_attributes_columns.buildUpPlayPassingClass, 
        team_attributes_columns.buildUpPlayPositioningClass,
        team_attributes_columns.chanceCreationPassing, 
        team_attributes_columns.chanceCreationPassingClass, 
        team_attributes_columns.chanceCreationCrossing, 
        team_attributes_columns.chanceCreationCrossingClass,
        team_attributes_columns.chanceCreationShooting, 
        team_attributes_columns.chanceCreationShootingClass, 
        team_attributes_columns.chanceCreationPositioningClass, 
        team_attributes_columns.defencePressure,
        team_attributes_columns.defencePressureClass, 
        team_attributes_columns.defenceAggression, 
        team_attributes_columns.defenceAggressionClass, 
        team_attributes_columns.defenceTeamWidth, 
        team_attributes_columns.defenceTeamWidthClass, 
        team_attributes_columns.defenceDefenderLineClass
    FROM team
    INNER JOIN team_attributes_columns
        on team.team_fifa_api_id = cast(team_attributes_columns.team_fifa_api_id as integer)
""")

cursor.executescript('DROP TABLE IF EXISTS team_attributes_columns;')

#Modelo da tabela Player_Attributes_Modified
tbl_name : str = 'team_attributes_modified'
rootpage : int = 10
sql : str = 'CREATE TABLE'
schema_update(schema_type, name, tbl_name, rootpage, sql)

#Criando em a tabela Match_Modified
cursor.execute(""" 
            CREATE TABLE match_modified as
            SELECT  '{"id":'|| id ||','||
                    '"match_api_id":'|| match_api_id ||','||
                    '"home_team_api_id":'|| home_team_api_id ||','||
                    '"away_team_api_id":'|| away_team_api_id ||'}'
            FROM match 
            """)

#Finalizando a conexão
conn.commit()
conn.close()